# Расчет AUC-ROC и AUC-PR(C)

In [9]:
import numpy as np
import sklearn as skl
from sklearn.metrics import *
import pandas as pd

создадим функцию, которая для заданого порога по вектору вероятности выдает конечные значения класса 
(расчет только для бинарных случаев)

In [4]:
def thres(th, pr): # th - пороговое значение, pr - вектор вероятности принадлежности к классу
    proba=np.zeros((len(pr),1))
    for i in range(0,len(pr)):
        if pr[i]<th:
            proba[i]=0
        else: 
            proba[i]=1
        #print(th, pr[i], proba[i])
    return proba  # возвращает вектор только с 0 или 1

Функция AUC-PR(C)

In [72]:
# Y_test - истинные значения
# Y_pred - вектор вероятностей
# thr - порог, задавать в сотых, десятых долях единицы, для выбора шага порога при расчета точек графика
def auc_prc( Y_test, Y_pred, thr): 
    area = 0 # площадь
    prc_i, prc = 0, 0 # текущее и предыдущие значения prc, ось Y
    rec_i, rec = 0, 0 # текущее и предыдущее значения rec, ось X
    
    # с заданным порогом вычисляем точки графика AUC-PR(C)
    for i in range(0 , int(1/thr)):
        
        a = thres(0.01+i*thr, Y_pred) # значения вектора вероятностей при определенном пороге
        prc = precision_score(Y_test, a) #ось Y
        rec = recall_score(Y_test,a)     #ось X
        
        # S = S прямоугольника + S треугольника, образованных координатами 
        if (prc > 0 and prc_i >0): 
            area +=  np.abs(rec-rec_i)*min(prc_i,prc)+0.5*np.abs(rec-rec_i)*np.abs(prc-prc_i)
   
        prc_i=prc
        rec_i=rec
        #print(prc, rec,area)
    return area

Функция AUC-ROC

In [74]:
# Y_test - истинные значения
# Y_pred - вектор вероятностей
# thr - порог, задавать в сотых, десятых долях единицы, для выбора шага порога при расчета точек графика
def auc_roc( Y_test, Y_pred, thr):
    area = 0  # площадь
    tpr_i, tpr = 0, 0  # текущее и предыдущие значения tpr, ось Y
    fpr_i, fpr = 0, 0  # текущее и предыдущее значения rec, ось X

    for i in range(0 , int(1/thr)):
        a = thres(0.01+i*thr, Y_pred)  # оценка вероятности события при данном пороге вектора
        
        # расчет fpr при заданном пороге
        fp, tn = 0, 0
        prdct = thres(thr, Y_pred)
        y_t = np.array(Y_test)
        for i in range(0, len(prdct)):   
            #print(i, y_t[i], prdct[i])
            if ( int(y_t[i] == 0) and int(a[i] == 1)):
                fp += 1      
            elif ((y_t[i] == 0) & (a[i] == 0)):
                tn += 1
        #print(fp , tn)    
        fpr = fp/(fp+tn)  # ось X
        #print(fpr)
        # end fpr count     
        
        tpr = recall_score(Y_test,a)     #ось Y
        
        # S = S прямоугольника + S треугольника, образованных координатами    
        if (tpr > 0 and tpr_i >0):
            area += np.abs(fpr-fpr_i)*min(tpr_i,tpr)+0.5*np.abs(fpr-fpr_i)*np.abs(tpr-tpr_i)
   
        tpr_i=tpr
        fpr_i=fpr
        #print(prc, rec,area)
    return area

Проверим созданые функции на кейсе Affair (просматривали на одном из занятий)

In [11]:
data = pd.read_csv('affair_data.csv')
data.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,1
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,1
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,1
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,1


In [12]:
# модель предсказывает вероятность интрижки
from sklearn.linear_model import LogisticRegression

In [14]:
X  = pd.get_dummies(data, columns=['occupation','occupation_husb','religious'])
X.head(2)

,rate_marriage,age,yrs_married,children,educ,affair,occupation_1.0,occupation_2.0,occupation_3.0,occupation_4.0,...,occupation_husb_1.0,occupation_husb_2.0,occupation_husb_3.0,occupation_husb_4.0,occupation_husb_5.0,occupation_husb_6.0,religious_1.0,religious_2.0,religious_3.0,religious_4.0
0,3.0,32.0,9.0,3.0,17.0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1,3.0,27.0,13.0,3.0,14.0,1,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0


In [17]:
del X['affair']

In [24]:
Y = data['affair']
X.head(2)

,rate_marriage,age,yrs_married,children,educ,occupation_1.0,occupation_2.0,occupation_3.0,occupation_4.0,occupation_5.0,...,occupation_husb_1.0,occupation_husb_2.0,occupation_husb_3.0,occupation_husb_4.0,occupation_husb_5.0,occupation_husb_6.0,religious_1.0,religious_2.0,religious_3.0,religious_4.0
0,3.0,32.0,9.0,3.0,17.0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,3.0,27.0,13.0,3.0,14.0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0


In [25]:
model = LogisticRegression()

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [27]:
model.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
predictions = model.predict_proba(X_test)

In [33]:
predictions

array([[ 0.3541677 ,  0.6458323 ],
       [ 0.92404711,  0.07595289],
       [ 0.72946542,  0.27053458],
       ..., 
       [ 0.56092307,  0.43907693],
       [ 0.81398702,  0.18601298],
       [ 0.73570251,  0.26429749]])

Нужен второй столбец, отвечает за affair = 1

# Сравнение

AUC-ROC сравнение средства sklearn и ручная программа

In [82]:
%%time
# встроенные стредства sklear для площади AUC-ROC
sk = roc_auc_score(Y_test, predictions[:,1])
print('sklearn AUC-ROC area:', format(sk))

sklearn AUC-ROC area: 0.7460201461334318
Wall time: 0 ns


In [83]:
%%time
# написанная вручную программа
my = auc_roc(Y_test, predictions[:,1], 0.01)
print('my AUC-ROC area:', format(my))

my AUC-ROC area: 0.7458399764325387
Wall time: 1.04 s


In [85]:
#ошибка
print('ошибка: ', format (abs(sk - my)))

ошибка:  0.0001801697008930736


AUC-PRC сравнение средства sklearn и ручная программа

In [87]:
%%time
# встроенные стредства sklear для площади AUC-PRC
sk = average_precision_score(Y_test, predictions[:,1])
print('sklearn AUC-PRC area:', format(sk))

sklearn AUC-PRC area: 0.5763261907254331
Wall time: 0 ns


In [88]:
%%time
# написанная вручную программа
my = auc_prc(Y_test, predictions[:,1], 0.01)
print('my AUC-PRC area:', format(my))

my AUC-PRC area: 0.5737588482446592
Wall time: 255 ms


C:\Users\blacat\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [89]:
#ошибка
print('ошибка: ', format (abs(sk - my)))

ошибка:  0.0025673424807739487


Значения площадей близки, функции работают корректно.